# Aufgaben Blatt 4 KI Machine Learning I



## Aufgabe 1 (Lineare Regression)

Bearbeiten Sie die Aufgabe https://github.com/oduerr/ki/blob/main/linear_regression/lr_gradient_descent.ipynb

Versuchen Sie den Code zu verstehen und machen die kleineren Aufgaben, die in dem notebook besprochen werden.

## Aufgabe 2 (Titanic)
In dieser Aufgabe nehmen Sie an der Titanic Challenge (https://www.kaggle.com/c/titanic) teil. Sie können die Aufgabe am eigenen PC lösen oder direkt in Kaggle lösen. Die Daten liegen auch auf Moodle. 

a) Lesen Sie die Trainingsdaten ein und teilen Sie sie in ein Validierungsdatenset (20%) und in ein eigentliches Trainigsdatenset (80%) auf. Finden Sie auf dem Trainigsdatenset eine Regel für das Überleben alleine aufgrund der Klasse des Tickets (Pclass). Wenden Sie diese Regel auf die Validierungsdaten an. Wie gut ist die Genauigkeit (Anteil der korrekten Klassifikationen) auf den Validierungsdaten?  

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

train_val = pd.read_csv('titanic/train.csv')

train_data, val_data = train_test_split(train_val, test_size=0.2, random_state=1) #random state damit das shuffeling reproduzierbar ist

# Hinweise zum Erzeugen einer Tabelle
# pd.crosstab(...)
ctab = pd.crosstab(train_data['Pclass'], train_data['Survived'], normalize='index')
print(ctab)

# Hinweise um die Accuracy zu berechnen
from sklearn.metrics import accuracy_score

#Regel: wenn Überlebenschance der Klasse > 0.5, dann gilt, dass vermutlicherweise alle Personen dieser Klasse überleben

class_survival_rate = ctab[1]

def predict_survival(pclass):
    if class_survival_rate[pclass] >= 0.5:
        return 1
    else:
        return 0

val_predictions = val_data['Pclass'].apply(predict_survival)

accuracy = accuracy_score(val_data['Survived'], val_predictions)
print(f"Genauigkeit der Vorhersagen: {accuracy:.2f}")

Survived         0         1
Pclass                      
1         0.373563  0.626437
2         0.513333  0.486667
3         0.775773  0.224227
Genauigkeit der Vorhersagen: 0.66


b) Wenden Sie die Regel aus a) auf die Testdaten an und laden Sie Ihre Lösung hoch. 

In [18]:
test_data = pd.read_csv('titanic/test.csv')
pred_survived = test_data['Pclass'].apply(predict_survival)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': pred_survived})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!


c) Logistische Regression mit Pclass

Trainieren Sie eine logistische Regression mit den Variablen 'Pclass'. Verwenden Sie die Klasse `sklearn.linear_model.LogisticRegression`. Berechnen Sie die Accuracy auf dem Validierungsset.

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

x_train = train_data[['Pclass']]
y_train = train_data['Survived']

x_val = val_data[['Pclass']]
y_val = val_data['Survived']

#Model trainieren
model = LogisticRegression()
model.fit(x_train, y_train)

#trainiertes Model Vorhersagen treffen lassen
val_predictions = model.predict(x_val)

#Genauigkeit der Vorhersagen berechnen
accuracy = accuracy_score(y_val, val_predictions)

print(f"Genauigkeit: {accuracy:.7f}")


Genauigkeit: 0.6592179


d) Coding / Feature engineering 

d.i) Missing Values:

Verwenden Sie nun weitere Features. Die Variable Age enthält Missing values, die Sie durch folgenden code ersetzen können (was passiert da?)

In [34]:
#val_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True)
#train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True)

#Berechnet den Median des Age-Werts im Trainingsdatensatz (ohne fehlende Werte zu berücksichtigen)
#Ersetzt alle fehlenden Werte (NaN) in der Spalte Age durch diesen Medianwert

age_median = train_data["Age"].median(skipna=True)

val_data["Age"] = val_data["Age"].fillna(age_median)
train_data["Age"] = train_data["Age"].fillna(age_median)

#Zwei Merkmale Pclass und Age
x_train = train_data[['Pclass', 'Age']]
y_train = train_data['Survived']

x_val = val_data[['Pclass', 'Age']]
y_val = val_data['Survived']

#Model trainieren
model = LogisticRegression()
model.fit(x_train, y_train)

# Vorhersage und Accuracy
predictions = model.predict(x_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Genauigkeit mit Pclass und Age: {accuracy:.7f}")

Genauigkeit mit Pclass und Age: 0.6648045


d.ii) Kategorische Variable

Verwenden Sie die Funktion `pd.get_dummies` um die Variablen 'Pclass' and 'Sex' in numerische Werte umzuwandeln. Führen Sie nun eine logistische Regression durch.

In [ ]:
#Umwandelung in numerische Werte
x_train_features = pd.get_dummies(train_data[['Pclass', 'Sex']], drop_first=True) #drop_first lässt ein Kategorie entfallen 
x_val_features = pd.get_dummies(val_data[['Pclass', 'Sex']], drop_first=True)

# Age hinzufügen (weil es numerische Werte sind, werden diese nachhinein hinzugefügt)
x_train_features['Age'] = train_data['Age']
y_train = train_data['Survived']

x_val_features['Age'] = val_data['Age']
y_val = val_data['Survived']

model = LogisticRegression()
model.fit(x_train_features, y_train)

val_predictions = model.predict(x_val_features)

accuracy = accuracy_score(y_val, val_predictions)
print(f"Genauigkeit: {accuracy:.7f}")

Genauigkeit: 0.8044693


e) Weitere Klassifikatoren. Neben der logistischen Regression, gibt es weitere Klassifikatoren. Der Random-Forest ist ein recht stabiler Klassifikator, was wäre die Performance von diesem Klassifikator.

In [51]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(x_train_features, y_train)
val_predictions = rf.predict(x_val_features)

accuracy = accuracy_score(y_val, val_predictions)
print(f"Random Forest Accuracy: {accuracy:.7f}")

Random Forest Accuracy: 0.7765363


f) [optional] Versuchen Sie weitere Features zu erzeugen und laden den besten Klassifikator auf Kaggle hoch. 

## Aufgabe 3 Titanic mit Neuronalen Netzen 

Hinweis: Diese Aufgabe kann erst nach der dritten Vorlesung in ML gemacht werden.

Mit den gleichen Daten, wie in der Aufgabe 2 d. Erstellen Sie ein fully connected neural network und fitten es an die Ttrainingsdaten. Verwenden Sie mindestens zwei hidden Layer. Plotten Sie den Verlauf der Loss Kurve für die Trainings- und Validierungsdaten. Optional: Laden Sie Ihre beste Lösung auf Kaggle hoch. 

In [ ]:
#Sie können von folgendem Code starten um das Netzwerk zu definieren, füllen Sie die ...
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(..., activation='sigmoid', batch_input_shape=(None, 4))) #We have 4 input features
#...
model.add(Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________
